# GeoPandas空间分析-上

GeoPandas提供了丰富的空间分析能力，该部分主要讲解单个GeoSeries/GeoDataFrame的空间分析操作

https://geopandas.org/en/stable/docs/reference/geoseries.html#constructor

In [ ]:
from shapely import geometry
import geopandas as gpd
import pyproj

In [ ]:
zjg=gpd.read_file('data/zjg.geojson')
zjg.plot()

## 通用方法和属性

这部分的内容在Ex09里有涉及，此处再回顾一下：

### area

In [ ]:
zjg['area']=zjg.to_crs(crs='EPSG:32651').area
zjg

### bounds

In [ ]:
zjg.bounds

### length

In [ ]:
zjg['length']=zjg.to_crs(crs='EPSG:32651').length
zjg

### geom_type

In [ ]:
zjg.geom_type

### x/y/z

In [ ]:
print(zjg.representative_point().x)
print(zjg.representative_point().y)

## 构造方法和属性

这类方法通过已有的几何对象建立新的对象

### boundary

维度降一级：多边形→线环，线→点

In [ ]:
%matplotlib widget
zjg.boundary.plot()

### centroid

In [ ]:
ax = zjg.to_crs(crs='EPSG:32651').boundary.plot()
zjg.to_crs(crs='EPSG:32651').centroid.plot(ax=ax, color='Red')

### representative_point()

初略计算（可以拿之前的全国区划数据自己试一下和centroid的用时差距）

In [ ]:
ax = zjg.to_crs(crs='EPSG:32651').boundary.plot()
zjg.to_crs(crs='EPSG:32651').representative_point().plot(ax=ax, color='Red')

### convex_hull / envelope

分别返回凸包和aabbox

In [ ]:
east=zjg.loc[zjg['name']=='East']
ax=east.envelope.plot(alpha=0.2)
east.convex_hull.plot(ax=ax,alpha=0.4,color='Yellow')
east.boundary.plot(ax=ax,color='Red')

### buffer()

缓冲区分析是最重要的空间分析方法之一，GeoPandas提供了buffer方法用于缓冲区分析

In [ ]:
east = zjg.loc[zjg['name']=='East'].to_crs(crs='EPSG:32651')
ax = east.plot()
east.buffer(100).plot(ax=ax,alpha=0.2)    # 单位是米

`buffer`的第二个参数`resolution`可以控制缓冲区转角的圆滑程度，`resolution=1`时可以生成直角边的缓冲

In [ ]:
ax = east.plot()
east.buffer(100,1).plot(ax=ax,alpha=0.2)     

距离设置为负值时可向内缓冲

In [ ]:
ax = east.buffer(-100,1).plot()
east.plot(ax=ax,alpha=0.2)  

### simplify()

返回简化的几何对象,`tolerance`参数确保简化对象的每一部分与原始对象的距离均小于该值（单位取决于坐标系）

In [ ]:
ax = east.boundary.plot()
east.simplify(0.0005).plot(ax=ax,alpha=0.6,color='Red')

## 聚合与拆分

GeoPandas提供了一系列聚合与拆分方法，用于将多个几何对象聚合为一个，或将一个几何对象拆分为多个

### unary_union

该属性返回全部几何对象合并成的一个几何对象

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.plot(edgecolor='white',linewidth=0.5)

In [ ]:
world.unary_union

In [ ]:
# unary_union返回的是一个几何对象，而不是GeoSeries/GeoDataFrame
type(world.unary_union)

### dissolve()

`dissolve()`方法可以对GeoDataFrame中的几何对象按照一定的字段规则进行聚合

In [ ]:
world.head()

In [ ]:
world_diss = world.loc[:,['pop_est','continent','gdp_md_est','geometry']]
world_diss = world_diss.dissolve(by='continent',
                                 aggfunc={'pop_est':'sum',
                                          'gdp_md_est':'sum'})
world_diss.head()

In [ ]:
world_diss.plot(edgecolor='Yellow',linewidth=0.5)

### explode()

`explode()`方法能将一个`Multi-`类型的几何对象拆成多个

In [ ]:
world.explode(ignore_index=False, index_parts=True).head()

# GeoPandas空间分析-下

GeoPandas提供了丰富的空间分析能力，该部分主要讲解多个GeoSeries/GeoDataFrame之间的空间分析操作

## 拓扑关系

拓扑关系是几何对象间最基本的关系，GeoPandas提供了许多用于判断拓扑关系的方法。
需要注意的是，GeoSeries/GeoDataframe可能含有一个或多个几何对象，因此需要进行区分，以`A.Topo(B)`为例，可能包含如下情况：

- A有n个对象，B有n个对象：A中的每一个几何对象会按索引和B中的对象两两对比，返回一个长为n的Series
- A有n个对象，B有m个对象（n≠m）：A中的每一个几何对象会按索引和B中的对象两两对比，多出的行自动返回False

### contains()

In [ ]:
gs_a = gpd.GeoDataFrame({
    'value': ['a1', 'a2'],
    'geometry': [geometry.Polygon([(0,0), (2,0), (2,2), (0,2)]),
                 geometry.Polygon([(0,0), (-1,0), (-1,-1), (0,-1)]),]
})


gs_b = gpd.GeoDataFrame({
    'value': ['b1', 'b2'],
    'geometry': [geometry.Polygon([(1,1), (1,1.5), (1.5,1.5), (1.5,1)]),
                 geometry.Polygon([(0,0), (-1,0), (-1,-1), (0,-1)])]
})

ax = gs_a.plot(color='Red', edgecolor='Yellow', alpha=0.4)
ax = gs_b.plot(color='Blue', edgecolor='Yellow', alpha=0.4, ax=ax)

gs_a.contains(gs_b)

### within()

In [ ]:
gs_a = gpd.GeoDataFrame({
    'value': ['a1', 'a2'],
    'geometry': [geometry.Polygon([(0,0), (2,0), (2,2), (0,2)]),
                 geometry.Polygon([(0,0), (-1,0), (-1,-1), (0,-1)]),]
})


gs_b = gpd.GeoDataFrame({
    'value': ['b1', 'b2'],
    'geometry': [geometry.Polygon([(1,1), (1,1.5), (1.5,1.5), (1.5,1)]),
                 geometry.Polygon([(0,0), (-1,0), (-1,-1), (0,-1)])]
})

ax = gs_a.plot(color='Red', edgecolor='Yellow', alpha=0.4)
ax = gs_b.plot(color='Blue', edgecolor='Yellow', alpha=0.4, ax=ax)

gs_a.within(gs_b)

### intersects()

In [ ]:
gs_a = gpd.GeoDataFrame({
    'value': ['a1', 'a2', 'a3', 'a4'],
    'geometry': [geometry.Polygon([(0,0), (2,0), (2,2), (0,2)]),
                 geometry.Polygon([(0,0), (-2,0), (-2,2), (0,2)]),
                 geometry.Polygon([(0,0), (-1,0), (-1,-1), (0,-1)]),
                 geometry.Polygon([(0,0), (1,0), (1,-1), (0,-1)])]
})

gs_b = gpd.GeoDataFrame({
    'value': ['b1', 'b2', 'b3', 'a4'],
    'geometry': [geometry.Polygon([(1,1), (1,3), (3,3), (3,1)]),
                 geometry.Polygon([(0,0), (-1,0), (-1,1), (0,1)]),
                 geometry.Polygon([(0,0), (-1,0), (-1,-1), (0,-1)]),
                 geometry.Polygon([(2,-1), (2,-2), (1,-2), (1,-1)])]
})

ax = gs_a.plot(color='Red', edgecolor='Yellow', alpha=0.4)
ax = gs_b.plot(color='Blue', edgecolor='Yellow', alpha=0.4, ax=ax)

gs_a.intersects(gs_b)

### disjoint()

In [ ]:
gs_a = gpd.GeoDataFrame({
    'value': ['a1', 'a2', 'a3', 'a4'],
    'geometry': [geometry.Polygon([(0,0), (2,0), (2,2), (0,2)]),
                 geometry.Polygon([(0,0), (-2,0), (-2,2), (0,2)]),
                 geometry.Polygon([(0,0), (-1,0), (-1,-1), (0,-1)]),
                 geometry.Polygon([(0,0), (1,0), (1,-1), (0,-1)])]
})

gs_b = gpd.GeoDataFrame({
    'value': ['b1', 'b2', 'b3', 'a4'],
    'geometry': [geometry.Polygon([(1,1), (1,3), (3,3), (3,1)]),
                 geometry.Polygon([(0,0), (-1,0), (-1,1), (0,1)]),
                 geometry.Polygon([(0,0), (-1,0), (-1,-1), (0,-1)]),
                 geometry.Polygon([(2,-1.1), (2,-2.1), (1,-2.1), (1,-1.1)])]    #向下移动0.1
})   
ax = gs_a.plot(color='Red', edgecolor='Yellow', alpha=0.4)
ax = gs_b.plot(color='Blue', edgecolor='Yellow', alpha=0.4, ax=ax)

gs_a.disjoint(gs_b)

### crosses()

In [ ]:
gs_a = gpd.GeoDataFrame({
    'value': ['a1', 'a2', 'a3', 'a4'],
    'geometry': [geometry.Polygon([(0,0), (2,0), (2,2), (0,2)]),
                 geometry.LineString([(0,0), (-2,2)]),
                 geometry.Polygon([(0,0), (-1,0), (-1,-1), (0,-1)]),
                 geometry.Polygon([(0,0), (1,0), (1,-1), (0,-1)])]
})

gs_b = gpd.GeoDataFrame({
    'value': ['b1', 'b2', 'b3', 'a4'],
    'geometry': [geometry.LineString([(1,1), (3,3)]),
                 geometry.LineString([(-3,-1), (0,3)]),
                 geometry.LineString([(-2,-1),(-1,-2)]),
                 geometry.Point(2,-2)]
})

ax = gs_a.plot(color='Red', edgecolor='Yellow', alpha=0.4)
ax = gs_b.plot(color='Blue', edgecolor='Yellow', alpha=0.4, ax=ax)

gs_a.crosses(gs_b)

### overlaps()

内部有重叠，但不是全部重叠

In [ ]:
gs_a = gpd.GeoDataFrame({
    'value': ['a1', 'a2', 'a3', 'a4'],
    'geometry': [geometry.Polygon([(0,0), (2,0), (2,2), (0,2)]),
                 geometry.Polygon([(0,0), (-2,0), (-2,2), (0,2)]),
                 geometry.Polygon([(0,0), (-1,0), (-1,-1), (0,-1)]),
                 geometry.Polygon([(0,0), (1,0), (1,-1), (0,-1)])]
})

gs_b = gpd.GeoDataFrame({
    'value': ['b1', 'b2', 'b3', 'a4'],
    'geometry': [geometry.Polygon([(1,1), (1,3), (3,3), (3,1)]),
                 geometry.Polygon([(0,0), (-1,0), (-1,1), (0,1)]),
                 geometry.Polygon([(0,0), (-1,0), (-1,-1), (0,-1)]),
                 geometry.Polygon([(2,-1), (2,-2), (1,-2), (1,-1)])]
})

ax = gs_a.plot(color='Red', edgecolor='Yellow', alpha=0.4)
ax = gs_b.plot(color='Blue', edgecolor='Yellow', alpha=0.4, ax=ax)

gs_a.overlaps(gs_b)

### touches()

内部没有相交区域

In [ ]:
gs_a = gpd.GeoDataFrame({
    'value': ['a1', 'a2', 'a3'],
    'geometry': [geometry.Polygon([(0,0), (2,0), (2,2), (0,2)]),
                 geometry.Polygon([(0,0), (-1,0), (-1,-1), (0,-1)]),
                 geometry.Polygon([(0,0), (1,0), (1,-1), (0,-1)])]
})

gs_b = gpd.GeoDataFrame({
    'value': ['b1', 'b2', 'b3'],
    'geometry': [geometry.Polygon([(1,1), (1,3), (3,3), (3,1)]),
                 geometry.Polygon([(-1,-0.5), (-2,-0.5), (-2,-1.5), (-1,-1.5)]),
                 geometry.Polygon([(2,-1), (2,-2), (1,-2), (1,-1)])]
})

ax = gs_a.plot(color='Red', edgecolor='Yellow', alpha=0.4)
ax = gs_b.plot(color='Blue', edgecolor='Yellow', alpha=0.4, ax=ax)

gs_a.touches(gs_b)

### geom_equals()

In [ ]:
gs_a = gpd.GeoDataFrame({
    'value': ['a1', 'a2', 'a3', 'a4'],
    'geometry': [geometry.Polygon([(0,0), (2,0), (2,2), (0,2)]),
                 geometry.Polygon([(0,0), (-2,0), (-2,2), (0,2)]),
                 geometry.Polygon([(0,0), (-1,0), (-1,-1), (0,-1)]),
                 geometry.Polygon([(0,0), (1,0), (1,-1), (0,-1)])]
})

gs_b = gpd.GeoDataFrame({
    'value': ['b1', 'b2', 'b3', 'a4'],
    'geometry': [geometry.Polygon([(1,1), (1,3), (3,3), (3,1)]),
                 geometry.Polygon([(0,0), (-1,0), (-1,1), (0,1)]),
                 geometry.Polygon([(0,0), (-1,0), (-1,-1), (0,-1)]),
                 geometry.Polygon([(2,-1), (2,-2), (1,-2), (1,-1)])]
})

ax = gs_a.plot(color='Red', edgecolor='Yellow', alpha=0.4)
ax = gs_b.plot(color='Blue', edgecolor='Yellow', alpha=0.4, ax=ax)

gs_a.geom_equals(gs_b)

### 逐行对应（`align=False`）

`align=False`能让两个GeoDataFrame比较时按行顺序匹配，不按索引名匹配。

![img](https://geopandas.org/en/stable/_images/binary_op-02.svg)

In [ ]:
gs_a = gpd.GeoDataFrame({
    'value': ['1', '2', '3', '4'],
    'geometry': [geometry.Polygon([(0,0), (2,0), (2,2), (0,2)]),
                 geometry.Polygon([(0,0), (-2,0), (-2,2), (0,2)]),
                 geometry.Polygon([(0,0), (-1,0), (-1,-1), (0,-1)]),
                 geometry.Polygon([(0,0), (1,0), (1,-1), (0,-1)])]
})
gs_a.set_index('value',inplace=True)

gs_b = gpd.GeoDataFrame({
    'value': ['1', '4', '2', '3'],
    'geometry': [geometry.Polygon([(1,1), (1,3), (3,3), (3,1)]),
                 geometry.Polygon([(-1,1), (-1,2), (-2,2), (-2,1)]),
                 geometry.Polygon([(-1,-0.5), (-2,-0.5), (-2,-1.5), (-1,-1.5)]),
                 geometry.Polygon([(2,-1), (2,-2), (1,-2), (1,-1)])]
})
gs_b.set_index('value',inplace=True)

ax = gs_a.plot(color='Red', edgecolor='Yellow', alpha=0.4)
ax = gs_b.plot(color='Blue', edgecolor='Yellow', alpha=0.4, ax=ax)

In [ ]:
print(gs_a)
print(gs_b)

In [ ]:
gs_a.intersects(gs_b, align=False)

In [ ]:
gs_a.intersects(gs_b, align=True)

## 叠加分析

GeoPandas的`overlay()`方法提供了对GeoDataFrame对象进行叠加分析的能力

In [ ]:
polys1 = gpd.GeoSeries([geometry.Polygon([(0,0), (2,0), (2,2), (0,2)]),
                              geometry.Polygon([(2,2), (4,2), (4,4), (2,4)])])
polys2 = gpd.GeoSeries([geometry.Polygon([(1,1), (3,1), (3,3), (1,3)]),
                              geometry.Polygon([(3,3), (5,3), (5,5), (3,5)])])
df1 = gpd.GeoDataFrame({'geometry': polys1, 'df1_data':[1,2]})
df2 = gpd.GeoDataFrame({'geometry': polys2, 'df2_data':[1,2]})

In [ ]:
df1

In [ ]:
df2

In [ ]:
ax = df1.plot(color='blue',alpha=0.5)
df2.plot(ax=ax, color='yellow', alpha=0.5)

### union

In [ ]:
overlay_union = gpd.overlay(df1,df2,how='union')
overlay_union

In [ ]:
overlay_union.plot(edgecolor='Yellow')

### intersection

In [ ]:
overlay_inter = gpd.overlay(df1,df2,how='intersection')
overlay_inter

In [ ]:
ax=overlay_inter.plot(edgecolor='Yellow')
ax.set_xlim(-0.2,5.2)
ax.set_ylim(-0.2,5.2)

### difference

In [ ]:
overlay_diff = gpd.overlay(df1,df2,how='difference')
overlay_diff

In [ ]:
ax = overlay_diff.plot(edgecolor='yellow')
ax.set_xlim(-0.2,5.2)
ax.set_ylim(-0.2,5.2)

In [ ]:
# difference方法实际上是区分主次的
overlay_diff = gpd.overlay(df2,df1,how='difference')
overlay_diff

In [ ]:
ax = overlay_diff.plot(edgecolor='yellow')
ax.set_xlim(-0.2,5.2)
ax.set_ylim(-0.2,5.2)

In [ ]:
# 为了更直接的表达主次关系，也可以使用geodataframe.overlaye()的形式：
overlay_diff = df1.overlay(df2,how='difference')
ax = overlay_diff.plot(edgecolor='yellow')
ax.set_xlim(-0.2,5.2)
ax.set_ylim(-0.2,5.2)

### symmetric_difference

交集取反

In [ ]:
overlay_symdiff = gpd.overlay(df1,df2,how='symmetric_difference')
overlay_symdiff

In [ ]:
ax = overlay_symdiff.plot(edgecolor='yellow')
ax.set_xlim(-0.2,5.2)
ax.set_ylim(-0.2,5.2)

### identity

标识

In [ ]:
road = gpd.GeoSeries([geometry.LineString([(0,0), (4,1), (5,5)]),
                      geometry.LineString([(1,0), (4,4), (0,5)])])
area = gpd.GeoSeries([geometry.Polygon([(0,0), (3,0), (3,5), (0,5)]),
                      geometry.Polygon([(3,0), (5,0), (5,5), (3,5)])])
df_road = gpd.GeoDataFrame({'geometry': road, 'df1_data':['road_1','road_2']})
df_area = gpd.GeoDataFrame({'geometry': area, 'df2_data':['area_1','area_2']})

In [ ]:
ax = df_road.plot(color='yellow')
df_area.plot(ax=ax, color='green', edgecolor='white', alpha=0.5)

In [ ]:
df_iden = gpd.overlay(df_road, df_area, how='identity')

df_iden

In [ ]:
ax = df_iden.loc[df_iden['df2_data']=='area_1'].plot(color='red')
df_iden.loc[df_iden['df2_data']=='area_2'].plot(ax = ax, color='blue')
df_area.plot(ax=ax, color='green', edgecolor='white', alpha=0.5)

## 裁切与提取

通过`clip()`函数，我们可以实现用某一蒙版矢量来裁切另一个地理对象

In [ ]:
road = gpd.GeoSeries([geometry.LineString([(0,0), (4,1), (5,5)]),
                      geometry.LineString([(1,0), (4,4), (0,5)])])
area = gpd.GeoSeries([geometry.Polygon([(2,0), (2,5), (3.5,5), (3.5,0)])])

df_road = gpd.GeoDataFrame({'geometry': road, 'df1_data':['road_1','road_2']})
df_area = gpd.GeoDataFrame({'geometry': area, 'df2_data':['area_1']})

ax = df_road.plot(color='blue')
df_area.plot(ax=ax, color='green', edgecolor='white', alpha=0.5)

In [ ]:
df_clip = gpd.clip(df_road, df_area)
df_clip

In [ ]:
ax = df_clip.plot(color='blue')
df_area.plot(ax=ax, color='green', edgecolor='white', alpha=0.5)
ax.set_xlim(-0.2,5.2)
ax.set_ylim(-0.2,5.2)

需要注意clip方法和overlay方法的不同：后者是独立计算每一个结合对象的。

因此在df2有部分重叠的情况下，overlay方法可能会导致df1中的几何对象被重复提取，以下是一个具体例子：

In [ ]:
road = gpd.GeoSeries([geometry.LineString([(0,0), (4,1), (5,5)]),
                      geometry.LineString([(1,0), (4,4), (0,5)])])
area = gpd.GeoSeries([geometry.Polygon([(2,0), (2,5), (3.5,5), (3.5,0)]),
                      geometry.Polygon([(0,0), (5,0), (5,1), (0,1)])])

df_road = gpd.GeoDataFrame({'geometry': road, 'df1_data':['road_1','road_2']})
df_area = gpd.GeoDataFrame({'geometry': area, 'df2_data':['area_1','area_2']})

ax = df_road.plot(color='blue')
df_area.plot(ax=ax, color='green', edgecolor='white', alpha=0.5)

In [ ]:
df_clip = gpd.clip(df_road, df_area)
df_clip

In [ ]:
# 如果是intersection
df_road
df_road.set_geometry('geometry')

In [ ]:
df_inter = df_road.overlay(df_area,how='intersection')
df_inter 

In [ ]:
# 可以看到一共有四条记录，我们画出来看一下，注意两个区域的重叠范围（绿框）内的情况
ax = df_area.iloc[[0]].overlay(df_area.iloc[[1]],how='intersection').boundary.plot(color='green', edgecolor='white', alpha=0.4)
df_inter.loc[df_inter['df2_data']=='area_1'].plot(ax=ax, color='red',alpha=1)    # 与area1重叠的部分
df_inter.loc[df_inter['df2_data']=='area_2'].plot(ax=ax, color='blue',alpha=0.6) # 与area2重叠的部分

# 空间连接

空间连接是非常重要的一个能力，在实际应用中使用到的情况也非常多，因此在此单列一节

## 空间数据和非空间数据的属性连接

空间数据和非空间数据之间可以通过彼此的属性进行连接，这种方法在Ex10b的练习中应该已经使用过了，GeoPandas继承了Pandas的`Merge`, `Join`等方法，因此可以像Pandas中操作两个普通数据表一样进行属性连接

### merge

In [ ]:
zhejiang = gpd.read_file('data/zhejiang.geojson')
zhejiang.head()

In [ ]:
pop = gpd.read_file('data/zhejiang_pop.csv')
pop.drop('geometry',axis=1,inplace=True)
pop.head()

In [ ]:
zhejiang_pop = zhejiang.merge(pop,left_on='市',right_on='地级市')
zhejiang_pop.head()

In [ ]:
pop.rename(columns={'地级市':'市'},inplace=True)
pop.head()

In [ ]:
# 如果两个Dataframe具有相同的列名，在不指定left_on和right_on参数的情况下会自动以相同列作为merge的基准
zhejiang_pop = zhejiang.merge(pop)
zhejiang_pop.head()

#### `how`参数

该参数有`left`, `right`, `outer`, `inner`四种值，用于指定在两边数据不能完全对齐情况下的连接方法

In [ ]:
zhejiang_a = zhejiang.loc[zhejiang['市'].isin(['杭州市','宁波市','温州市','嘉兴市','湖州市'])]
zhejiang_a

In [ ]:
pop_b = pop.loc[pop['市'].isin(['杭州市','宁波市','温州市','嘉兴市'])]
pop_b

In [ ]:
left_merge = zhejiang_a.merge(pop_b,how='left')
left_merge

In [ ]:
right_merge = zhejiang_a.merge(pop_b,how='right')
right_merge

In [ ]:
outer_merge = zhejiang_a.merge(pop_b,how='outer')
outer_merge

In [ ]:
outer_merge2 = pop_b.merge(zhejiang_a,how='outer')
outer_merge2 

In [ ]:
inner_merge = zhejiang_a.merge(pop_b,how='inner')
inner_merge

In [ ]:
inner_merge2 = pop_b.merge(zhejiang_a,how='inner')
inner_merge2 

### join

In [ ]:
zhejiang.set_index('市',inplace=True)
zhejiang.head()

In [ ]:
pop.set_index('市',inplace=True)
pop.head()

In [ ]:
# 如果两个Dataframe的index可以作为连接的基准，那么可以直接使用join()方法
zhejiang.join(pop).head()

## 空间数据之间的连接

`sjoin` 方法可以通过拓扑关系来实现空间连接，具体的拓扑关系由参数 `predicate` (旧版本使用`op`) 指定

In [ ]:
# 比如我们目前手上有一份杭州外卖订单的分布数据（已脱敏）
express = gpd.read_file('data/express_data_sample.geojson', encoding='UTF-8')
hz = gpd.read_file('data/hangzhou.geojson')
hz=hz.loc[hz['类型']=='市辖区']
ax = hz.plot(alpha=0.5)
express.plot(ax=ax, markersize=1,color='yellow')

In [ ]:
# 要统计各市辖区之内的外卖订单数，用sjon可以很方便的完成
# sjoin将两份空间数据以intersects方法关联起来
sjoin_hz = hz.sjoin(df=express,predicate='intersects')
sjoin_hz.head()

In [ ]:
# 关联的结果是若干行一对一的数据, 我们想要他变成一对多的——使用group by实现
groupsjoin_hz = sjoin_hz.groupby('NAME').agg({'geometry':'first','id': 'count'}).rename(columns={'id': 'exp_count'})
groupsjoin_hz

In [ ]:
groupsjoin_hz.plot(edgecolor='grey',linewidth=0.5,column='exp_count',cmap='GnBu')